In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

In [6]:
root_data_dir = "../Datasets/NER/BiodivNER/"

dataset = "train"
train_csv_file_path = "train.csv"
val_csv_file_path = "dev.csv"
test_csv_file_path = "test.csv"

In [7]:
def loadData(csv_file_path):
  dataset_path = os.path.join(root_data_dir, csv_file_path)
  data = pd.read_csv(dataset_path, encoding="latin1")
  data = data.fillna(method="ffill")
  return data

In [8]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),                                                          
                                                        s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [9]:
data = loadData(train_csv_file_path)

In [10]:
val_data = loadData(val_csv_file_path)

In [11]:
test_data = loadData(test_csv_file_path)

In [12]:
VOCAB = list(set(list(data["Word"].values) + \
                 list(val_data["Word"].values) + \
                 list(test_data["Word"].values)))
VOCAB.append("ENDPAD")

n_words = len(VOCAB) #n_words includes all vocab from train and validation test.

tags = list(set(data["Tag"].values))

n_tags = len(tags)

In [13]:
getter = SentenceGetter(data)
sentences = getter.sentences
sent = getter.get_next()
print(sent)

[('bottom', 'O'), ('board', 'O'), ('debris', 'O'), (',', 'O'), ('frames', 'O'), (',', 'O'), ('landing', 'O'), ('platforms', 'O'), (')', 'O'), (',', 'O'), ('and', 'O'), ('isolates', 'O'), ('of', 'O'), ('microbes', 'O'), (',', 'O'), ('parasites', 'O'), ('and', 'O'), ('pathogens', 'O'), ('from', 'O'), ('honey', 'B-Organism'), ('bees', 'I-Organism'), ('.', 'O')]


In [14]:
getter_val = SentenceGetter(val_data)
sentences_val = getter_val.sentences
sent_val = getter_val.get_next()
print(sent_val)

[('However', 'O'), (',', 'O'), ('the', 'O'), ('lack', 'O'), ('of', 'O'), ('correlation', 'O'), ('between', 'O'), ('dung', 'O'), ('beetle', 'O'), ('community', 'B-Environment'), ('characteristics', 'O'), ('and', 'O'), ('dung', 'O'), ('removal', 'O'), ('highlights', 'O'), ('the', 'O'), ('need', 'O'), ('for', 'O'), ('further', 'O'), ('research', 'O'), ('into', 'O'), ('spatial', 'O'), ('variation', 'O'), ('in', 'O'), ('biodiversityÃ\x83Â¢Ã¢Â\x82Â¬Ã¢Â\x80Â\x9cecosystem', 'O'), ('function', 'O'), ('relationships', 'O'), ('and', 'O'), ('how', 'O'), ('the', 'O'), ('results', 'O'), ('of', 'O'), ('such', 'O'), ('studies', 'O'), ('are', 'O'), ('affected', 'O'), ('by', 'O'), ('methodological', 'O'), ('choices', 'O'), ('.', 'O')]


In [15]:
getter_test = SentenceGetter(test_data)
sentences_test = getter_test.sentences
sent_test = getter_test.get_next()
print(sent_test)

[('The', 'O'), ('primacy', 'O'), ('of', 'O'), ('either', 'O'), ('species', 'B-Quality'), ('or', 'O'), ('functional', 'O'), ('group', 'O'), ('richness', 'B-Quality'), ('effects', 'O'), ('depended', 'O'), ('on', 'O'), ('the', 'O'), ('sequence', 'O'), ('of', 'O'), ('testing', 'O'), ('these', 'O'), ('terms', 'O'), (',', 'O'), ('indicating', 'O'), ('that', 'O'), ('both', 'O'), ('aspects', 'O'), ('of', 'O'), ('richness', 'B-Quality'), ('were', 'O'), ('congruent', 'O'), ('and', 'O'), ('complementary', 'O'), ('to', 'O'), ('expected', 'O'), ('strong', 'O'), ('effects', 'O'), ('of', 'O'), ('legume', 'O'), ('presence', 'O'), ('and', 'O'), ('grass', 'B-Organism'), ('presence', 'O'), ('on', 'O'), ('plant', 'B-Organism'), ('chemical', 'B-Quality'), ('composition', 'I-Quality'), ('.', 'O')]


In [16]:
tag2id = {tag: id for id, tag in enumerate(tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [17]:
tag2id_list = list(tag2id.items())

In [18]:
id2tag_list = list(id2tag.items())

In [19]:
def get_text_tags_lists(sentences):
  texts = []
  tags = []
  for sent in sentences: #list of tuples    
    sent_texts = []
    sent_tags = []  
    for tuple1 in sent:  
      sent_texts.append(tuple1[0])
      sent_tags.append(tuple1[1])

    texts.append(sent_texts)
    tags.append(sent_tags)
  return texts, tags

In [20]:
train_texts, train_tags = get_text_tags_lists(sentences)
val_texts, val_tags = get_text_tags_lists(sentences_val)
test_texts, test_tags = get_text_tags_lists(sentences_test)

In [21]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW, Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [22]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
def convert_iob2_to_t5(words, tags):
    t5_labels = []

    for word, tag in zip(words, tags):
        if tag.startswith('B-'):
            t5_labels.append(f"<{tag[2:]}> {word}")
        elif tag.startswith('I-'):
            t5_labels.append(word)
        else:
            t5_labels.append(f"</{tag[2:]}>")

    return ' '.join(t5_labels)

In [24]:
train_t5_inputs = [convert_iob2_to_t5(words, tags) for words, tags in zip(train_texts, train_tags)]
val_t5_inputs = [convert_iob2_to_t5(words, tags) for words, tags in zip(val_texts, val_tags)]
test_t5_inputs = [convert_iob2_to_t5(words, tags) for words, tags in zip(test_texts, test_tags)]

In [25]:
def tokenize_data(input_texts):
    input_ids = []
    attention_masks = []

    for text in input_texts:
        encoded = tokenizer.encode_plus(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

In [26]:
X_train, attention_masks_train = tokenize_data(train_t5_inputs)
X_val, attention_masks_val = tokenize_data(val_t5_inputs)
X_test, attention_masks_test = tokenize_data(test_t5_inputs)

In [34]:
def get_text_tags_lists_512(sentences, max_length=512):
  texts = []
  tags = []
  for sent in sentences: #list of tuples    
    sent_texts = []
    sent_tags = []  
    for tuple1 in sent:  
      word, tag = tuple1
      truncated_word = word[:max_length]
      truncated_tag = tag[:max_length]
      
      sent_texts.append(truncated_word)
      sent_tags.append(truncated_tag)

    texts.append(sent_texts)
    tags.append(sent_tags)
  return texts, tags

In [35]:
y_train, _ = get_text_tags_lists_512(sentences)
y_val, _ = get_text_tags_lists_512(sentences_val)
y_test, _ = get_text_tags_lists_512(sentences_test)

In [36]:
def convert_labels_to_ids(labels):
    label_ids = [tokenizer.encode(label, return_tensors='pt')[0] for label in labels]
    return torch.cat(label_ids, dim=0)

In [37]:
y_train_ids = convert_labels_to_ids(y_train)
y_val_ids = convert_labels_to_ids(y_val)
y_test_ids = convert_labels_to_ids(y_test)

In [38]:
train_dataset = TensorDataset(X_train, attention_masks_train, y_train_ids)
val_dataset = TensorDataset(X_val, attention_masks_val, y_val_ids)
test_dataset = TensorDataset(X_test, attention_masks_test, y_test_ids)

batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

AssertionError: Size mismatch between tensors

In [39]:
print("X_train size:", X_train.size())
print("attention_masks_train size:", attention_masks_train.size())
print("y_train_ids size:", y_train_ids.size())

X_train size: torch.Size([1918, 512])
attention_masks_train size: torch.Size([1918, 512])
y_train_ids size: torch.Size([81666])


In [43]:
print(y_train)
print(y_train_ids)

[['Samplenr', 'Seedlingnr', 'Plot', 'Record', 'Date', 'Planted_Species', 'Density', 'Treatment', 'Dead', 'Height_P', 'Height_G', 'Leaves_Liv', 'Leaves_Dam', 'Leaves_Dead', 'Damage_pro', 'Biomass_Above', 'Biomass_Below', 'List', 'of', 'headers', 'of', 'the', 'data', 'columns', 'in', 'this', 'dataset', 'Pilot', 'experiment', '117.8998', '118.1483', '29.2852', '29.10178', '########', '########', 'markus_ger', 'erfmeier', 'Common', 'Garden', 'Experiment', ':', 'Seedling', 'addition', 'experiment', '-', 'growth', 'and', 'biomass', 'data', 'While', 'coexistence', 'in', 'plant', 'communities', 'is', 'frequently', 'explained', 'by', 'effects', 'of', 'resource', 'niche', 'partitioning', ',', 'the', 'Janzen-Connell', '(', 'J-C', ')', 'hypothesis', 'is', 'an', 'alternative', 'approach', 'that', 'has', 'been', 'assumed', 'as', 'a', 'major', 'ecological', 'mechanism', 'explaining', 'high', 'species', 'richness', 'levels', ',', 'in', 'particular', ',', 'in', 'tropical', 'forest', 'ecosystems', '.'],